# ChemCluster Project Report

**CH-200 – Practical Programming in Chemistry**  
**Group:** Elisa Rubbia, Romain Guichonnet, Flavia Zabala Perez  
**Date:** May 2025


## Welcome to ChemCluster!

ChemCluster is a streamlined and interactive platform designed to facilitate the analysis and visualization of chemical datasets using molecular clustering techniques. It leverages the power of **RDKit** for cheminformatics and **Streamlit** for web interface deployment, offering an intuitive solution for researchers and students alike.

Two main modes of operation are supported by the platform:

- **Dataset Mode**: This mode allows users to upload datasets (in `.csv` or `.sdf` format) containing multiple molecular structures. These are then clustered based on structural similarity, and representative structures for each cluster are highlighted for further exploration.
- **Single Molecule Mode**: In this workflow, ChemCluster generates a set of 3D conformers for a single input molecule. These conformers are clustered, and representative structures are visualized both in 2D and 3D, enabling users to analyze conformational diversity effectively.

## Setup and Initialization

To begin using ChemCluster, you have two options: installation via PyPI or running the application locally from source.

### Installation from PyPI
The simplest way to install ChemCluster is through the Python Package Index. In a terminal or command prompt, run:

```bash
pip install chemcluster
```
Once installed, you can launch the application by executing:

```bash
chemcluster
```
This command will open the ChemCluster interface directly in your default web browser.

### Running Locally from Source (for Development or Contribution)
If you prefer to contribute to the project or wish to run it locally in a development environment, follow these steps:
1. Clone the repository from GitHub:
```bash
git clone https://github.com/erubbia/ChemCluster
```
2. Navigate into the project directory:
```bash
cd ChemCluster
```
3. Create a conda environment based on the project's environment file:
```bash
conda env create -f environment.yml
```
4. Activate the newly created environment:
```bash
conda activate chemcluster-env
```
5. Finally, install the project in editable mode:
```bash
pip install -e .
```
After this setup, you can launch ChemCluster the same way by running `chemcluster` in the terminal.